In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [8]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[460000:]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_460.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_460.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 460000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

460000: 0:00:00.008997 elapsed
461000: 0:16:05.142585 elapsed
462000: 0:33:01.870811 elapsed
463000: 0:50:59.211558 elapsed
464000: 1:06:45.922743 elapsed
465000: 1:23:21.800415 elapsed
466000: 1:41:57.974685 elapsed
467000: 1:58:43.838046 elapsed
468000: 2:15:35.174405 elapsed
469000: 2:33:51.876673 elapsed
470000: 2:51:57.042120 elapsed
471000: 3:08:15.111683 elapsed
472000: 3:25:22.817500 elapsed
473000: 3:41:50.360771 elapsed
474000: 4:00:51.478144 elapsed


In [9]:
with open(data_path + "images_saved_460.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_460.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [10]:
len(images_saved)

3827